# Crypto RL Portfolio (PPO) — Continuous Weights

In [ ]:

CONFIG = {
    "DATA": {
        "drive_folder_id": "1uXEBUyySypdsW_ZqL-RZ3d1bWdIZisij",
        "structure": {"1d":"ohlcv_1d","1h":"ohlcv_1h","15m":"ohlcv_15m","5m":"ohlcv_5m","1m":"ohlcv_1m"},
        "file_pattern": "{TICKER}_{FREQ}.parquet",
        "tickers": ["BTC","ETH"],
        "sampling": "1h",
        "price_point": "close",
        "timezone_in": "CET",
        "timezone_out": "UTC",
        "forward_fill": True,
        "drop_na_after_ffill": True,
        "cache_dir": "./data_cache",
        "DOWNLOAD": {
            "method": "folder",
            "file_ids": {},
            "download_all": False
        }
    },
    "ENV": {
        "action_space": "continuous_weights",
        "include_cash": True,
        "shorting": False,
        "max_leverage": 1.0,
        "rebalance_interval": "1_bar",
        "episode_length": {"mode":"steps","value":2160},
        "lookback_window": 64,
        "features": {
            "log_return_window": 1,
            "vol_window": 64,
            "rsi_period": 14,
            "volume_change": True,
            "scaler": "rolling_zscore"
        },
        "transaction_costs": {
            "commission_bps": 5.0,
            "slippage_bps": 5.0,
            "apply_on_rebalance_only": True
        },
        "turnover_penalty": 0.0,
        "weight_smoothing": 0.0,
        "reward": {"type":"log_return_minus_risk","risk_lambda":0.001},
        "constraints": {"min_weight":0.0,"max_weight":1.0,"sum_to_one":True},
        "seed": 42
    },
    "SPLITS": {
        "data_start":"2024-09-02",
        "data_end":"2025-09-02",
        "train":["2024-09-02","2025-06-15"],
        "val":["2025-06-16","2025-07-15"],
        "test":["2025-07-16","2025-09-02"],
        "walk_forward": True,
        "wf_train_span_days": 180,
        "wf_test_span_days": 30,
        "wf_step_days": 30
    },
    "RL": {
        "algo":"PPO",
        "timesteps":8_000_000,
        "device":"auto",
        "policy":"MlpPolicy",
        "gamma":0.99,
        "gae_lambda":0.95,
        "clip_range":0.2,
        "n_steps":2048,
        "batch_size":8192,
        "learning_rate":3e-4,
        "ent_coef":0.0,
        "vf_coef":0.5,
        "max_grad_norm":0.5
    },
    "EVAL": {
        "benchmarks":["equal_weight_hold","buy_and_hold_BTC","buy_and_hold_ETH"],
        "metrics":["CAGR","Sharpe","Sortino","MaxDrawdown","Calmar","Volatility","Turnover","HitRatio"],
        "plots":True,
        "reports_dir":"./reports"
    },
    "IO": {
        "models_dir":"./models",
        "tb_logdir":"./tb",
        "save_best_on_val": True
    }
}
CONFIG


In [ ]:

%pip -q install -U numpy pandas pyarrow gdown gymnasium stable-baselines3 torch matplotlib tqdm


In [ ]:

import os
import math
import json
import glob
import shutil
import random

from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

from tqdm import tqdm

def set_all_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    except Exception:
        pass

set_all_seeds(CONFIG["ENV"]["seed"])

ANNUALIZATION = {"1m":365*24*60,"5m":365*24*12,"15m":365*24*4,"1h":365*24,"1d":365}

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


In [ ]:

import gdown

ROOT_ID = CONFIG["DATA"]["drive_folder_id"]
CACHE_DIR = CONFIG["DATA"]["cache_dir"]
ensure_dir(CACHE_DIR)

def download_drive_folder(root_id: str, out_dir: str):
    print("Mirroring Google Drive folder locally...")
    gdown.download_folder(id=root_id, output=out_dir, quiet=False, use_cookies=False)

def targeted_download_by_ids(file_id_map: Dict[str, str], out_dir: str):
    ensure_dir(out_dir)
    for name, fid in file_id_map.items():
        suffix = name if name.endswith(".parquet") else f"{name}.parquet"
        out_path = os.path.join(out_dir, suffix)
        print(f"Downloading {name} -> {out_path}")
        url = f"https://drive.google.com/uc?id={fid}"
        gdown.download(url, out_path, quiet=False, use_cookies=False)

download_method = CONFIG["DATA"]["DOWNLOAD"]["method"]
if download_method == "file_ids" and CONFIG["DATA"]["DOWNLOAD"]["file_ids"]:
    targeted_download_by_ids(CONFIG["DATA"]["DOWNLOAD"]["file_ids"], CACHE_DIR)
else:
    download_drive_folder(ROOT_ID, CACHE_DIR)

print("Download step complete.")


In [ ]:

sampling = CONFIG["DATA"]["sampling"]
subfolder = CONFIG["DATA"]["structure"][sampling]
pattern_fmt = CONFIG["DATA"]["file_pattern"]

def find_parquet_path(ticker: str, sampling: str) -> str:
    fname = pattern_fmt.format(TICKER=ticker, FREQ=sampling)
    candidates = glob.glob(os.path.join(CACHE_DIR, "**", subfolder, fname), recursive=True)
    if not candidates:
        candidates = glob.glob(os.path.join(CACHE_DIR, "**", fname), recursive=True)
    if not candidates:
        raise FileNotFoundError(f"Could not find {fname} under {CACHE_DIR}.")
    return candidates[0]

def localize_and_align(df: pd.DataFrame, tz_in: str, tz_out: str) -> pd.DataFrame:
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp')
    if df.index.tz is None:
        df = df.tz_localize(tz_in)
    df = df.tz_convert(tz_out)
    cols = {c: c.lower() for c in df.columns}
    df = df.rename(columns=cols)
    return df.sort_index()

tz_in, tz_out = CONFIG["DATA"]["timezone_in"], CONFIG["DATA"]["timezone_out"]
tickers = CONFIG["DATA"]["tickers"]

dfs = {}
for t in tickers:
    pth = find_parquet_path(t, sampling)
    tmp = pd.read_parquet(pth)
    tmp = localize_and_align(tmp, tz_in, tz_out)
    if CONFIG["DATA"]["forward_fill"]:
        tmp = tmp.ffill()
    if CONFIG["DATA"]["drop_na_after_ffill"]:
        tmp = tmp.dropna()
    dfs[t] = tmp

common_index = None
for t, df in dfs.items():
    common_index = df.index if common_index is None else common_index.intersection(df.index)
for t in tickers:
    dfs[t] = dfs[t].reindex(common_index).dropna()

print({t: dfs[t].shape for t in tickers})
dfs["BTC"].head()


In [ ]:

def compute_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    up = (delta.clip(lower=0)).ewm(alpha=1/period, adjust=False).mean()
    down = (-delta.clip(upper=0)).ewm(alpha=1/period, adjust=False).mean()
    rs = up / (down + 1e-12)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def make_features(df: pd.DataFrame, price_col: str, vol_window: int, rsi_period: int, volume_change: bool):
    out = pd.DataFrame(index=df.index)
    out["ret"] = np.log(df[price_col]).diff(1)
    out["vol"] = out["ret"].rolling(vol_window).std().fillna(0.0)
    out["rsi"] = compute_rsi(df[price_col], rsi_period).fillna(50.0)
    if "volume" in df.columns and volume_change:
        out["volchg"] = np.log(df["volume"].replace(0, np.nan)).diff().fillna(0.0)
    else:
        out["volchg"] = 0.0
    return out

feat_cfg = CONFIG["ENV"]["features"]
lookback = CONFIG["ENV"]["lookback_window"]
price_col = CONFIG["DATA"]["price_point"]

features_by_ticker = {}
for t in tickers:
    fdf = make_features(dfs[t], price_col, feat_cfg["vol_window"], feat_cfg["rsi_period"], feat_cfg["volume_change"])
    features_by_ticker[t] = fdf

panel_cols = []
for t in tickers:
    for col in ["ret","vol","rsi","volchg"]:
        panel_cols.append((t, col))
panel = pd.concat([features_by_ticker[t][["ret","vol","rsi","volchg"]] for t in tickers], axis=1)
panel.columns = pd.MultiIndex.from_tuples(panel_cols, names=["ticker","feature"])
panel = panel.dropna()

print(panel.tail())
panel.describe()


In [ ]:

def rolling_zscore(df: pd.DataFrame, window: int = 256) -> pd.DataFrame:
    mu = df.rolling(window).mean()
    sigma = df.rolling(window).std().replace(0, np.nan)
    z = (df - mu) / (sigma + 1e-12)
    return z.fillna(0.0)

def build_state_tensor(panel: pd.DataFrame, lookback: int, scaler: str = "rolling_zscore"):
    if scaler == "rolling_zscore":
        scaled = panel.groupby(level=1, axis=1).apply(lambda g: rolling_zscore(g, window=max(lookback*2, 256)))
        scaled.columns = panel.columns
    else:
        scaled = panel.copy()

    tickers = sorted({c[0] for c in scaled.columns})
    features = sorted({c[1] for c in scaled.columns})
    times = scaled.index

    X, y_ret, inst_vol = [], [], []
    for i in range(lookback, len(times)-1):
        window_slice = scaled.iloc[i-lookback:i]
        frames = []
        for t in tickers:
            frames.append(window_slice[t].T.values)
        tensor = np.stack(frames, axis=0)
        X.append(tensor)
        nxt = panel.iloc[i+1]
        y_ret.append(np.array([nxt[(t, "ret")] for t in tickers], dtype=float))
        cur = panel.iloc[i]
        inst_vol.append(np.array([cur[(t, "vol")] for t in tickers], dtype=float))

    X = np.array(X, dtype=np.float32)
    y_ret = np.array(y_ret, dtype=np.float32)
    inst_vol = np.array(inst_vol, dtype=np.float32)
    return X, y_ret, inst_vol, tickers, features, times[lookback+1:]

X_all, R_all, VOL_all, TICKER_ORDER, FEAT_ORDER, TIME_INDEX = build_state_tensor(
    panel, lookback=CONFIG["ENV"]["lookback_window"], scaler=CONFIG["ENV"]["features"]["scaler"]
)
print("State tensor:", X_all.shape, "Returns:", R_all.shape, "InstVol:", VOL_all.shape)


In [ ]:

from datetime import timedelta

def date_slice_mask(times: pd.DatetimeIndex, start: str, end: str):
    return (times >= pd.Timestamp(start, tz=CONFIG["DATA"]["timezone_out"])) & (times <= pd.Timestamp(end, tz=CONFIG["DATA"]["timezone_out"]))

def build_splits(times: pd.DatetimeIndex, cfg: dict):
    s = CONFIG["SPLITS"]
    if not s["walk_forward"]:
        m_train = date_slice_mask(times, s["train"][0], s["train"][1])
        m_val   = date_slice_mask(times, s["val"][0], s["val"][1])
        m_test  = date_slice_mask(times, s["test"][0], s["test"][1])
        return [{"name":"BaseSplit","train":m_train,"val":m_val,"test":m_test}]
    else:
        start = pd.Timestamp(s["data_start"], tz=CONFIG["DATA"]["timezone_out"])
        end   = pd.Timestamp(s["data_end"], tz=CONFIG["DATA"]["timezone_out"])
        spans = []
        cur_train_start = start
        while True:
            train_end = cur_train_start + timedelta(days=s["wf_train_span_days"])
            test_end  = train_end + timedelta(days=s["wf_test_span_days"])
            if test_end > end:
                break
            m_train = (times >= cur_train_start) & (times <= train_end)
            m_val   = (times > train_end) & (times <= train_end)
            m_test  = (times > train_end) & (times <= test_end)
            spans.append({"name":f"WF_{cur_train_start.date()}_{(test_end.date())}","train":m_train,"val":m_val,"test":m_test})
            cur_train_start = cur_train_start + timedelta(days=s["wf_step_days"])
        return spans

SPLITS = build_splits(TIME_INDEX, CONFIG["SPLITS"])
print(f"Built {len(SPLITS)} split(s). Example:", SPLITS[0]["name"] if SPLITS else "None")


In [ ]:

class PortfolioWeightsEnv(gym.Env):
    metadata = {"render_modes": []}

    def __init__(self, X, R, VOL, tickers, lookback, cfg_env, sampling="1h"):
        super().__init__()
        self.X = X
        self.R = R
        self.VOL = VOL
        self.tickers = tickers
        self.lookback = lookback
        self.cfg = cfg_env
        self.sampling = sampling

        self.n_assets = len(tickers)
        self.include_cash = cfg_env["include_cash"]
        self.dim_action = self.n_assets + (1 if self.include_cash else 0)

        obs_dim = self.n_assets * self.X.shape[2] * self.lookback
        self.observation_space = spaces.Box(low=-10, high=10, shape=(obs_dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(self.dim_action,), dtype=np.float32)

        self.commission = cfg_env["transaction_costs"]["commission_bps"] / 1e4
        self.slippage = cfg_env["transaction_costs"]["slippage_bps"] / 1e4
        self.apply_on_rebalance_only = cfg_env["transaction_costs"]["apply_on_rebalance_only"]
        self.risk_lambda = cfg_env["reward"]["risk_lambda"]

        self.min_w = cfg_env["constraints"]["min_weight"]
        self.max_w = cfg_env["constraints"]["max_weight"]
        self.sum_to_one = cfg_env["constraints"]["sum_to_one"]

        self.reset(seed=cfg_env.get("seed", 42))

    def _to_obs(self, t):
        arr = self.X[t].reshape(-1).astype(np.float32)
        return arr

    def _project_weights(self, a):
        if self.sum_to_one:
            expo = np.exp(a - np.max(a))
            w = expo / np.sum(expo)
        else:
            w = np.clip(a, self.min_w, self.max_w)
        if not self.cfg["shorting"]:
            w = np.clip(w, 0.0, 1.0)
        return w

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.t = 0
        self.portfolio_value = 1.0
        self.w = np.ones(self.dim_action) / self.dim_action
        obs = self._to_obs(self.t)
        return obs, {}

    def step(self, action):
        w_target = self._project_weights(action)
        turnover = np.sum(np.abs(w_target - self.w))
        trading_cost = (self.commission + self.slippage) * turnover

        asset_w_prev = self.w[:self.n_assets]
        asset_ret = np.dot(asset_w_prev, self.R[self.t])
        inst_vol = np.dot(asset_w_prev, self.VOL[self.t])

        reward = asset_ret - trading_cost - self.risk_lambda * inst_vol

        self.portfolio_value *= math.exp(asset_ret - trading_cost)

        self.w = w_target
        self.t += 1
        terminated = (self.t >= len(self.R)-1)
        truncated = False

        obs = self._to_obs(self.t) if not terminated else self._to_obs(self.t-1)
        info = {"portfolio_value": self.portfolio_value, "turnover": turnover, "inst_vol": inst_vol, "asset_ret": asset_ret}
        return obs, reward, terminated, truncated, info


In [ ]:

def slice_by_mask(X, R, VOL, mask: np.ndarray):
    idx = np.where(mask)[0]
    return X[idx], R[idx], VOL[idx]

def make_env_from_mask(mask, name="env"):
    X_s, R_s, V_s = slice_by_mask(X_all, R_all, VOL_all, mask)
    env = PortfolioWeightsEnv(X_s, R_s, V_s, TICKER_ORDER, CONFIG["ENV"]["lookback_window"], CONFIG["ENV"], sampling=CONFIG["DATA"]["sampling"])
    env = Monitor(env, filename=None)
    return env

print("Env factory ready.")


In [ ]:

def annualize_factor(sampling: str):
    return ANNUALIZATION.get(sampling, 365*24)

def compute_metrics(equity_curve: pd.Series, sampling: str, turnover_series: pd.Series = None):
    ret = equity_curve.pct_change().dropna()
    ann = annualize_factor(sampling)
    mu = ret.mean() * ann
    sigma = ret.std() * math.sqrt(ann)
    sharpe = mu / (sigma + 1e-12)
    downside = ret[ret < 0].std() * math.sqrt(ann)
    sortino = mu / (downside + 1e-12)
    if len(equity_curve) > 1:
        dt_years = (equity_curve.index[-1] - equity_curve.index[0]) / pd.Timedelta(days=365)
        dt_years = float(dt_years) if float(dt_years) != 0 else 1e-12
        cagr = (equity_curve.iloc[-1] / equity_curve.iloc[0]) ** (1/dt_years) - 1
    else:
        cagr = 0.0
    cummax = equity_curve.cummax()
    dd = (equity_curve / cummax - 1).min()
    maxdd = float(dd)
    calmar = mu / (abs(maxdd) + 1e-12)
    hit_ratio = (ret > 0).mean()
    turnover = turnover_series.mean() if turnover_series is not None and len(turnover_series)>0 else np.nan
    return {"CAGR": cagr, "Sharpe": sharpe, "Sortino": sortino, "MaxDrawdown": maxdd, "Calmar": calmar, "Volatility": sigma, "Turnover": turnover, "HitRatio": hit_ratio}

def plot_series(series: pd.Series, title: str):
    plt.figure(figsize=(10,4))
    plt.plot(series.index, series.values)
    plt.title(title); plt.xlabel("Time"); plt.ylabel("Value"); plt.show()

def backtest_env(env: PortfolioWeightsEnv, model=None):
    obs, _ = env.reset()
    pv, turns = [], []
    for t in range(len(env.R)-1):
        if model is None:
            action = np.ones(env.dim_action)/env.dim_action
        else:
            action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, trunc, info = env.step(action)
        pv.append(info["portfolio_value"])
        turns.append(info["turnover"])
        if done:
            break
    idx = pd.RangeIndex(start=0, stop=len(pv), step=1)
    ec = pd.Series(pv, index=idx)
    to = pd.Series(turns, index=idx)
    return ec, to


In [ ]:

ensure_dir(CONFIG["IO"]["models_dir"])
ensure_dir(CONFIG["EVAL"]["reports_dir"])

RESULTS = []

for split in SPLITS:
    print(f"\n=== Training on split: {split['name']} ===")
    train_env = make_env_from_mask(split["train"], name=f"{split['name']}_train")
    eval_env  = make_env_from_mask(split["test"], name=f"{split['name']}_test")

    vec_train = DummyVecEnv([lambda: train_env])
    vec_eval  = DummyVecEnv([lambda: eval_env])

    model = PPO(
        policy=CONFIG["RL"]["policy"],
        env=vec_train,
        gamma=CONFIG["RL"]["gamma"],
        gae_lambda=CONFIG["RL"]["gae_lambda"],
        clip_range=CONFIG["RL"]["clip_range"],
        n_steps=CONFIG["RL"]["n_steps"],
        batch_size=CONFIG["RL"]["batch_size"],
        learning_rate=CONFIG["RL"]["learning_rate"],
        ent_coef=CONFIG["RL"]["ent_coef"],
        vf_coef=CONFIG["RL"]["vf_coef"],
        max_grad_norm=CONFIG["RL"]["max_grad_norm"],
        tensorboard_log=CONFIG["IO"]["tb_logdir"],
        device=CONFIG["RL"]["device"],
        verbose=1
    )

    eval_callback = EvalCallback(vec_eval, best_model_save_path=CONFIG["IO"]["models_dir"],
                                 log_path=CONFIG["IO"]["models_dir"], eval_freq=10000,
                                 deterministic=True, render=False)
    model.learn(total_timesteps=CONFIG["RL"]["timesteps"], callback=eval_callback)
    model_path = os.path.join(CONFIG["IO"]["models_dir"], f"ppo_{split['name']}.zip")
    model.save(model_path)
    print("Saved model:", model_path)

    test_env = make_env_from_mask(split["test"], name=f"{split['name']}_test")
    ec, to = backtest_env(test_env, model=model)

    idx = np.where(split["test"])[0]
    R_test = R_all[idx]
    ew = np.ones(len(TICKER_ORDER))/len(TICKER_ORDER)
    ec_bench = [1.0]
    for r in R_test[:-1]:
        ec_bench.append(ec_bench[-1]*math.exp(np.dot(ew, r)))
    ec_bench = pd.Series(ec_bench, index=ec.index)

    bh_btc, bh_eth = [1.0], [1.0]
    for r in R_test[:-1]:
        bh_btc.append(bh_btc[-1]*math.exp(r[0]))
        bh_eth.append(bh_eth[-1]*math.exp(r[1]))
    bh_btc = pd.Series(bh_btc, index=ec.index)
    bh_eth = pd.Series(bh_eth, index=ec.index)

    m_model = compute_metrics(ec, CONFIG["DATA"]["sampling"], to)
    m_ew    = compute_metrics(ec_bench, CONFIG["DATA"]["sampling"])
    m_btc   = compute_metrics(bh_btc, CONFIG["DATA"]["sampling"])
    m_eth   = compute_metrics(bh_eth, CONFIG["DATA"]["sampling"])

    RESULTS.append({"split": split["name"], "model": m_model, "equal_weight": m_ew, "buy_and_hold_BTC": m_btc, "buy_and_hold_ETH": m_eth})

    if CONFIG["EVAL"]["plots"]:
        plot_series(ec, f"Equity Curve — PPO ({split['name']})")
        plot_series((ec / ec.cummax()) - 1.0, f"Drawdown — PPO ({split['name']})")
        plot_series(ec_bench, f"Equity Curve — Equal-Weight Hold ({split['name']})")

print("Done. RESULTS collected.")


In [ ]:

rows = []
for res in RESULTS:
    row = {"split": res["split"]}
    for k, metrics in res.items():
        if k == "split":
            continue
        for mname, mval in metrics.items():
            row[f"{k}_{mname}"] = mval
    rows.append(row)

df_results = pd.DataFrame(rows)
df_results


In [ ]:

from datetime import datetime
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
ensure_dir(CONFIG["EVAL"]["reports_dir"])
out_json = os.path.join(CONFIG["EVAL"]["reports_dir"], f"metrics_{ts}.json")
out_csv  = os.path.join(CONFIG["EVAL"]["reports_dir"], f"metrics_{ts}.csv")
df_results.to_csv(out_csv, index=False)
with open(out_json, "w") as f:
    json.dump(RESULTS, f, indent=2)
print("Saved:", out_json, "and", out_csv)


**Notes:** Adjust fees for Hyperliquid; consider a separate validation set; extend features and risk controls as needed.